In [4]:
import pandas as pd
import numpy as np

In [2]:
school = pd.read_csv('../data/school.csv',encoding_errors='ignore')
subway = pd.read_csv('../data/subway_feature.csv')
bus = pd.read_csv('../data/bus_feature.csv')
address = pd.read_csv('../data/addresses.csv')

In [3]:
address.head()

,주소,경도,위도
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291


In [7]:
def add_poi_counts(address_df: pd.DataFrame,
                   poi_df: pd.DataFrame,
                   radii=(1, 3, 5, 10),
                   lat_col='위도',
                   lon_col='경도',
                   suffix='poi') -> pd.DataFrame:
    """
    주소 데이터프레임(address_df)과 POI 데이터프레임(poi_df)을 받아,
    각 반경(radii) 내 POI 개수를 주소 행별로 계산해
    f'{r}_{suffix}' 이름의 컬럼으로 추가합니다.

    - 좌표는 km 단위 하버사인 거리로 계산
    - 주소/POI의 위경도 결측은 자동 제외
    """
    # 유효 좌표만 남기기
    poi_valid = poi_df.dropna(subset=[lat_col, lon_col]).copy()
    addr_valid_mask = address_df[lat_col].notna() & address_df[lon_col].notna()

    if poi_valid.empty or not addr_valid_mask.any():
        # 결과 컬럼만 NaN으로 생성
        for r in radii:
            address_df[f'{r}_{suffix}'] = np.nan
        return address_df

    # 넘파이 배열로 변환 (라디안)
    addr_lat = np.deg2rad(address_df.loc[addr_valid_mask, lat_col].to_numpy())
    addr_lon = np.deg2rad(address_df.loc[addr_valid_mask, lon_col].to_numpy())
    poi_lat = np.deg2rad(poi_valid[lat_col].to_numpy())
    poi_lon = np.deg2rad(poi_valid[lon_col].to_numpy())

    # 하버사인 거리 행렬 계산 (단위: km)
    R = 6371.0088  # 지구 반경(km)
    dlat = addr_lat[:, None] - poi_lat[None, :]
    dlon = addr_lon[:, None] - poi_lon[None, :]
    a = np.sin(dlat / 2) ** 2 + np.cos(addr_lat)[:, None] * np.cos(poi_lat)[None, :] * np.sin(dlon / 2) ** 2
    dist = 2 * R * np.arcsin(np.sqrt(a))

    # 반경별 개수 계산 및 컬럼 채우기
    for r in radii:
        col = f'{r}_{suffix}'
        counts = (dist <= r).sum(axis=1).astype(int)
        address_df[col] = np.nan
        address_df.loc[addr_valid_mask, col] = counts

    return address_df


In [8]:
# 주소별 버스 수 컬럼 추가
address = add_poi_counts(
    address_df=address,
    poi_df=bus,
    radii=(1, 3, 5, 10),
    lat_col='위도',
    lon_col='경도',
    suffix='bus'
)

# 결과 확인
address.head()

,주소,경도,위도,1_bus,3_bus,5_bus,10_bus
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57.0,439.0,1059.0,3424.0
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77.0,500.0,1135.0,3771.0
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49.0,397.0,1221.0,3698.0
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62.0,434.0,1136.0,3651.0
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52.0,430.0,1059.0,3444.0


In [9]:
# 주소별 지하철 수 컬럼 추가
address = add_poi_counts(
    address_df=address,
    poi_df=subway,
    radii=(1, 3, 5, 10),
    lat_col='위도',
    lon_col='경도',
    suffix='sub'
)

# 결과 확인
address.head()

,주소,경도,위도,1_bus,3_bus,5_bus,10_bus,1_sub,3_sub,5_sub,10_sub
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57.0,439.0,1059.0,3424.0,0.0,14.0,46.0,156.0
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77.0,500.0,1135.0,3771.0,4.0,20.0,50.0,174.0
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49.0,397.0,1221.0,3698.0,3.0,26.0,61.0,173.0
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62.0,434.0,1136.0,3651.0,6.0,21.0,55.0,170.0
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52.0,430.0,1059.0,3444.0,1.0,16.0,50.0,164.0


In [10]:
# 주소별 초등학교 수 컬럼 추가
address = add_poi_counts(
    address_df=address,
    poi_df=school,
    radii=(1, 3, 5, 10),
    lat_col='위도',
    lon_col='경도',
    suffix='school'
)

# 결과 확인
address.head()

,주소,경도,위도,1_bus,3_bus,5_bus,10_bus,1_sub,3_sub,5_sub,10_sub,1_school,3_school,5_school,10_school
0,서울특별시 강남구 개포동 언주로 3,127.056841,37.476283,57.0,439.0,1059.0,3424.0,0.0,14.0,46.0,156.0,4.0,24.0,52.0,172.0
1,서울특별시 강남구 개포동 개포로 307,127.056014,37.483973,77.0,500.0,1135.0,3771.0,4.0,20.0,50.0,174.0,7.0,25.0,58.0,198.0
2,서울특별시 강남구 개포동 개포로109길 69,127.076626,37.496296,49.0,397.0,1221.0,3698.0,3.0,26.0,61.0,173.0,4.0,31.0,73.0,197.0
3,서울특별시 강남구 개포동 개포로 310,127.064582,37.486862,62.0,434.0,1136.0,3651.0,6.0,21.0,55.0,170.0,7.0,24.0,60.0,192.0
4,서울특별시 강남구 개포동 선릉로 7,127.062627,37.480291,52.0,430.0,1059.0,3444.0,1.0,16.0,50.0,164.0,5.0,25.0,56.0,182.0


In [ ]:
address.to_csv('address_with_poi.csv', index=False)